In [42]:
import pandas as pd
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import sparse
import pickle

In [43]:
data = pd.read_pickle('cleaned_data.pkl')

In [44]:
data.head()

,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny,cleaned_review,normalized_text
0,9yKzy9PApeiPPOUJEtnvkg,2011-01-26,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,review,rLtl8ZkDX5vH5nAx9C3q5Q,2,5,0,wife took birthday breakfast excellent weather...,wife took birthday breakfast excellent weather...
1,ZRJwVLyzEJq1VAihDhYiow,2011-07-27,IjZ33sJrzXqU-0X6U8NwyA,5,I have no idea why some people give bad review...,review,0a2KyEL0d3Yb1V6aivbIuQ,0,0,0,idea people give bad reviews place goes show p...,idea people give bad review place go show plea...
2,6oRAC4uyJCsJl1X0WZpVSA,2012-06-14,IESLBzqUCLdSzSqm0eCSxQ,4,love the gyro plate.]Rice is so good and I als...,review,0hT2KtfLiobPvh6cDC8JQg,0,1,0,love gyro plate rice good also dig candy selec...,love gyro plate rice good also dig candy selec...
3,_1QQZuf4zZOyFCvXc0o6Vg,2010-05-27,G-WvGaISbqqaMHlNnByodA,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",review,uZetl9T0NcROGOyFfughhg,1,2,0,rosie dakota love chaparral dog park convenien...,rosie dakota love chaparral dog park convenien...
4,6ozycU1RpktNG2-1BroVtw,2012-01-05,1uJFq2r5QfJG_6ExMRCaGw,5,General Manager Scott Petello is a good egg!!!...,review,vYmM4KTsC8ZfQBg-j5MWkw,0,0,0,general manager scott petello good egg go deta...,general manager scott petello good egg go deta...


In [45]:
len(data.business_id.unique())

4174

In [46]:
data = data.loc[:,['business_id','stars','normalized_text']]

In [47]:
data

,business_id,stars,normalized_text
0,9yKzy9PApeiPPOUJEtnvkg,5,wife took birthday breakfast excellent weather...
1,ZRJwVLyzEJq1VAihDhYiow,5,idea people give bad review place go show plea...
2,6oRAC4uyJCsJl1X0WZpVSA,4,love gyro plate rice good also dig candy selec...
3,_1QQZuf4zZOyFCvXc0o6Vg,5,rosie dakota love chaparral dog park convenien...
4,6ozycU1RpktNG2-1BroVtw,5,general manager scott petello good egg go deta...
5,-yxfBYGB6SEqszmxJxd97A,4,quiessence simply put beautiful full window ea...
6,zp713qNhx8d9KCJJnrw1xA,5,drop drive ate go back next day food good cute...
7,hW0Ne_HTHEAgGF1rAdmR-g,4,luckily travel far make connecting flight than...
8,wNUea3IXZWD63bbOQaOH-g,4,definitely come happy hour price amazing sake ...
9,nMHhuYan8e3cONo3PornJA,5,nobuo show unique talent everything menu caref...


In [48]:
df = data.groupby('business_id').agg({'normalized_text':(' ').join,'stars':'mean'}).reset_index()

In [49]:
df.head()

,business_id,normalized_text,stars
0,-0QBrNvhrPQCaeo7mTo0zQ,best shrimp burro hood make cringe thought goi...,4.000000
1,-0bUDim5OGuv8R0Qqq6J4A,place busy kid love pancake much food price,3.000000
2,-1N0Z3uM8xbxKS8XiAnaog,office space layout presentation get star cust...,1.000000
3,-34jE_5dujSWMIOBudQsiQ,dr dairiki terrific professional listens expla...,5.000000
4,-3WVw1TNQbPBzaKCaQQ1AQ,lunch today hearing colossal raf however walke...,3.333333


In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4174 entries, 0 to 4173
Data columns (total 3 columns):
business_id        4174 non-null object
normalized_text    4174 non-null object
stars              4174 non-null float64
dtypes: float64(1), object(2)
memory usage: 98.0+ KB


In [51]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
#df['normalized_text'] = metadata['normalized_text']
tfidf_matrix = tfidf.fit_transform(df['normalized_text'])


In [52]:
tfidf_matrix.shape

(4174, 25019)

In [53]:
type(tfidf_matrix)

scipy.sparse.csr.csr_matrix

In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4174 entries, 0 to 4173
Data columns (total 3 columns):
business_id        4174 non-null object
normalized_text    4174 non-null object
stars              4174 non-null float64
dtypes: float64(1), object(2)
memory usage: 98.0+ KB


In [55]:
user_query = 'pizza'
user_vector = tfidf.transform([user_query])

In [56]:
user_vector.shape

(1, 25019)

In [57]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim2 = linear_kernel(tfidf_matrix,user_vector)

In [58]:
len(cosine_sim2)

4174

In [59]:
similar = pd.DataFrame(cosine_sim2)
similar['business_id'] = df['business_id']
similar['stars'] = df['stars']
similar.head()

,0,business_id,stars
0,0.0,-0QBrNvhrPQCaeo7mTo0zQ,4.000000
1,0.0,-0bUDim5OGuv8R0Qqq6J4A,3.000000
2,0.0,-1N0Z3uM8xbxKS8XiAnaog,1.000000
3,0.0,-34jE_5dujSWMIOBudQsiQ,5.000000
4,0.0,-3WVw1TNQbPBzaKCaQQ1AQ,3.333333


In [60]:
similar.sort_values([0,'stars'],ascending=False).head(10)

,0,business_id,stars
585,0.783444,7t9VbwL5orIV0-DzrjU7_A,5.0
3822,0.659204,uYQo0oDXI5NEoMJNWXnx8Q,4.0
1846,0.655004,R2FuwuSjmyvzNWNfqzKUvw,4.5
3649,0.642877,s1IkTuTdWmPunObMizXxSg,5.0
152,0.633934,13lNc7KSuVHBrUybTR23qw,4.0
1359,0.632492,Jje3kP5CNasYDSpH0NPRGQ,4.0
3595,0.629722,r4AG1WXCRWkZKQctrcy9FA,3.8
458,0.617428,5weJ3DRKwvQ9claomkYq9w,3.0
1569,0.617115,Mh85-BDqDHlngT8G2ZnRWg,3.0
1566,0.615604,MfCS8VeJzf9QbxNImekuMA,2.0


In [61]:
similar = similar.sort_values([0],ascending=False).head(10)
similar = similar.sort_values(['stars'],ascending=False)
similar

,0,business_id,stars
585,0.783444,7t9VbwL5orIV0-DzrjU7_A,5.0
3649,0.642877,s1IkTuTdWmPunObMizXxSg,5.0
1846,0.655004,R2FuwuSjmyvzNWNfqzKUvw,4.5
3822,0.659204,uYQo0oDXI5NEoMJNWXnx8Q,4.0
152,0.633934,13lNc7KSuVHBrUybTR23qw,4.0
1359,0.632492,Jje3kP5CNasYDSpH0NPRGQ,4.0
3595,0.629722,r4AG1WXCRWkZKQctrcy9FA,3.8
458,0.617428,5weJ3DRKwvQ9claomkYq9w,3.0
1569,0.617115,Mh85-BDqDHlngT8G2ZnRWg,3.0
1566,0.615604,MfCS8VeJzf9QbxNImekuMA,2.0


## Testing Resturant Recommendations by viewing the review text

In [65]:
df['normalized_text'][585]

'best pizza place ever'

In [66]:
df['normalized_text'][3649]

'pizza venice hand best pizza valley mind got best bang buck xl three topping pizza deal get garlic crust buying pizza nirvana hurt owner incredibly friendly would even consider ordering terrible chain pizza get great pizza great price locally owned business'

In [67]:
def get_recommendations(user_query,tfidf_matrix):
    user_vector = tfidf.transform([user_query])
    cosine_sim2 = linear_kernel(your_matrix_back,user_vector)
    
    ## create a dataframe
    similar = pd.DataFrame(cosine_sim2)
    similar['business_id'] = df['business_id']
    similar['stars'] = df['stars']
    similar['Review'] = df['normalized_text']
    
    ## First sorting based on similarity score and then based on average rating
    similar = similar.sort_values([0],ascending=False).head(10)
    similar = similar.sort_values(['stars'],ascending=False)
    return similar.head(5)

In [68]:
get_recommendations('pizza',tfidf_matrix)

,0,business_id,stars,Review
585,0.783444,7t9VbwL5orIV0-DzrjU7_A,5.0,best pizza place ever
3649,0.642877,s1IkTuTdWmPunObMizXxSg,5.0,pizza venice hand best pizza valley mind got b...
1846,0.655004,R2FuwuSjmyvzNWNfqzKUvw,4.5,delicious food without long wait staff friendl...
3822,0.659204,uYQo0oDXI5NEoMJNWXnx8Q,4.0,always good pizza close home serve beer wine p...
152,0.633934,13lNc7KSuVHBrUybTR23qw,4.0,excellent selection unusual delicious pizza fa...


In [69]:
#sparse.save_npz("yourmatrix.npz", tfidf_matrix)
your_matrix_back = sparse.load_npz("yourmatrix.npz")

In [70]:
#pickle.dump(tfidf, open("tfidf.pickle", "wb"))
t = pickle.load(open("tfidf.pickle", "rb"))

/Users/rakesh.nimma/anaconda3/envs/NLP-Basics/lib/python3.7/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.20.1 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/rakesh.nimma/anaconda3/envs/NLP-Basics/lib/python3.7/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.20.1 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [71]:
#df.to_pickle('d.pkl')
d = pd.read_pickle('d.pkl')

In [72]:
def get_recommendations(user_query,your_matrix_back):
    user_vector = t.transform([user_query])
    cosine_sim2 = linear_kernel(your_matrix_back,user_vector)
    
    ## create a dataframe
    similar = pd.DataFrame(cosine_sim2)
    similar['business_id'] = d['business_id']
    similar['stars'] = d['stars']
    similar['Reviews'] = d['normalized_text']
    
    ## First sorting based on similarity score and then based on average rating
    similar = similar.sort_values([0],ascending=False).head(5)
    similar = similar.sort_values(['stars'],ascending=False)
    #d={}
    similar.drop(columns=[0],inplace=True)
    return similar.T.to_dict().values()

## Final Output 

In [73]:
list(get_recommendations('pizza',your_matrix_back))

[{'business_id': '7t9VbwL5orIV0-DzrjU7_A',
  'stars': 5.0,
  'Reviews': 'best pizza place ever'},
 {'business_id': 's1IkTuTdWmPunObMizXxSg',
  'stars': 5.0,
  'Reviews': 'pizza venice hand best pizza valley mind got best bang buck xl three topping pizza deal get garlic crust buying pizza nirvana hurt owner incredibly friendly would even consider ordering terrible chain pizza get great pizza great price locally owned business'},
 {'business_id': 'R2FuwuSjmyvzNWNfqzKUvw',
  'stars': 4.5,
  'Reviews': 'delicious food without long wait staff friendly accommodating look forward going back got craving pizza want feel guilty eating always seems satiate fix pizza ultra thin many variety available bryan always get pizza would still hungry order one without appetizer always take extra home today ate pepperoni mushroom pizza potato feta cheese pizza olive tempanade yummy way ate tomato mozzarella pesto bruschetta appetizer came unbelievably quick waiting pizza sauce pizza unique right crave want 

## Refrence

https://www.datacamp.com/community/tutorials/recommender-systems-python?utm_source=adwords_ppc&utm_campaignid=1565261270&utm_adgroupid=67750485268&utm_device=m&utm_keyword=&utm_matchtype=b&utm_network=g&utm_adpostion=1t1&utm_creative=295208661514&utm_targetid=dsa-473406569915&utm_loc_interest_ms=&utm_loc_physical_ms=9027290&gclid=Cj0KCQjwrrXtBRCKARIsAMbU6bENj2prGXE8WoQgLs7g7V-TDbmQTbH3I61sU2-XRWLJEQEiPLkCYf0aAl4QEALw_wcB